In [292]:
#[0]
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile
import json #https://stackoverflow.com/questions/7100125/storing-python-dictionaries
import pickle #https://stackoverflow.com/questions/11218477/how-can-i-use-pickle-to-save-a-dict
import os
#[1]
from sklearn.manifold import MDS
from scipy.spatial import procrustes
import matplotlib.pyplot as plt

%matplotlib inline

#[2]
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN

from sklearn import decomposition
from scipy.spatial import distance_matrix

In [295]:
EE = get_emotion_embedding()
sentiments = get_sentiments()

# Critical values for p

In [296]:
for k in range(15):
    real = EE.copy()
    distances_real = distance_matrix(real,real)
    iterations = 300
    trocas = k
    erros = []
    for i in range(iterations):
        mod = real.copy()
        for j in range(trocas):
            a,b = np.random.randint(len(sentiments),size=2)
            while(a==b):
                a,b = np.random.randint(len(sentiments),size=2)
            mod[a] = real[b]
            mod[b] = real[a]

        distance_mod = distance_matrix(mod,mod)
        erro = calcula_stress(distances_real,distance_mod)
        erros += [erro]
    print(k, np.mean(erros),np.var(erros),sorted(erros)[15],sorted(erros)[30])

0 0.0 0.0 0.0 0.0
1 0.007206784168071933 8.531319132103985e-06 0.003529259643742473 0.004077889816912012
2 0.010493993904163383 7.722654050352972e-06 0.0067742938131513655 0.007269970455829811
3 0.013032305569693717 8.324390667053905e-06 0.008912540869600954 0.009493568438700637
4 0.014916879664691983 6.793726174592676e-06 0.010940736564225645 0.011772404060213366
5 0.016616188568739976 8.433606275331054e-06 0.012070101605253892 0.012878501086318585
6 0.018120205915287603 8.328395844804186e-06 0.013720434953544825 0.014602217179944093
7 0.01954915468254717 8.057605260205343e-06 0.014791760988873621 0.015916044178740884
8 0.02067420131691701 7.844525315754382e-06 0.016293057187647262 0.017278844908838034
9 0.021855610382772304 7.669350593833649e-06 0.017938877379422808 0.018454158773923243
10 0.023134322437827384 7.363138188898518e-06 0.01875365460337248 0.01964638198659176
11 0.023803260421549746 7.432504672943391e-06 0.019429369874140748 0.02046548379122279
12 0.02505570654819777 7.69

# Stress values for each dimension

In [298]:
stress = []
x = []
for i in range(1,20):
    x+=[i]
    r =MDS(n_components=i,max_iter=3000,n_init=10).fit(EE)
    stress += [r.stress_]
    print(i,r.stress_,calcula_stress_embeddings(EE,r.embedding_))

1 83.84558729344081 0.13695836999799835
2 36.407061682481086 0.09023412080286992
3 20.690486457554655 0.06801252115058443
4 13.306478977054674 0.05453060242483441
5 9.957608270940174 0.047161616534743156
6 7.520384508599208 0.040975070611373184
7 6.165443858914251 0.03709167843166618
8 5.2125747391867785 0.03409711178561332
9 4.4871891955574 0.03162822073119787
10 3.8210499710157526 0.029177674652144023
11 3.3730273859289053 0.027406088169874082
12 3.0760863698653615 0.02616494889809195
13 2.8635127414983517 0.025239980511612282
14 2.528552949054131 0.023709584771149414
15 2.3690003447117602 0.02294413423523367
16 2.216295205642981 0.022187228271389303
17 2.019613124045688 0.021172239810941768
18 1.8992126390774864 0.02052671431376964
19 1.7343976486463668 0.019610176583303045


# Define quantidade stress-per-point

In [302]:
ssp = []
t = []
for i in range(len(sentiments)):
    a = (sentiments[i],get_stresses_per_point(i,EE,2))
    t += [a[1]]
    print(a)
    ssp += [a]

('astonished', -0.0002811271358145184)
('determined', 0.0006021000821617717)
('serene', 0.00013996985755006242)
('joyous', 0.0003365630752256987)
('desperate', 1.3458985313671845e-05)
('melancholic', 7.926144332501273e-05)
('amorous', 5.1061197400115166e-05)
('condescending', -0.0007848213515741359)
('distrustful', -7.543416364615141e-05)
('passionate', -0.0006331066308214311)
('dissatisfied', 0.0006040583542706313)
('empathic', -6.488590449835185e-05)
('lusting', 0.0005480167025402605)
('happy', -0.0003388567965483813)
('miserable', -0.00034444318402454543)
('indignant', 0.000362223875584905)
('jealous', 0.00027840556902411495)
('amused', -0.00030961339176370506)
('disgusted', -0.0003213735109613147)
('hopeful', -0.0005259340727305911)
('discontented', 0.00021058062486295492)
('sad', 0.0007664841014329571)
('insulted', 0.0006757980513041051)
('distressed', 0.00011503796922657239)
('tired', -0.0001305377834422694)
('suspicious', 9.898912573755303e-05)
('contemptuous', 0.000639218624369

In [305]:
sorted(ssp,key=lambda a: a[1],reverse=True)

[('guilt', 0.000892874424443621),
 ('worried', 0.0008542152881414466),
 ('satisfied', 0.0008337654291864227),
 ('sad', 0.0007664841014329571),
 ('calm', 0.0006849960157759988),
 ('insulted', 0.0006757980513041051),
 ('contemptuous', 0.0006392186243697334),
 ('dissatisfied', 0.0006040583542706313),
 ('determined', 0.0006021000821617717),
 ('interested', 0.0005750733929221208),
 ('lusting', 0.0005480167025402605),
 ('annoyed', 0.0005199938033915025),
 ('courageous', 0.0005032594760437242),
 ('wavering', 0.0004916235432469857),
 ('ambitious', 0.0004793345033042018),
 ('triumphant', 0.00043234448727982877),
 ('embarrassed', 0.0003829139366819667),
 ('depressed', 0.0003804757655897356),
 ('conceited', 0.0003675509251430814),
 ('indignant', 0.000362223875584905),
 ('joyous', 0.0003365630752256987),
 ('jealous', 0.00027840556902411495),
 ('discontented', 0.00021058062486295492),
 ('bitter', 0.00020236236612297676),
 ('frustrated', 0.00018864744201056716),
 ('serious', 0.00018036150922697003),

In [287]:
outliers_list = [10,5,24,39,7]

In [288]:
clean_EE = []
for a in range(len(EE)):
    if a not in outliers_list:
        clean_EE += [EE[a]]

In [301]:
def get_stresses_per_point(p, emb,d):
    stress_original = []
    stress_one_out = []

    for i in range(5):
        r =MDS(n_components=d,max_iter=3000,n_init=10).fit(emb)
        stress_original += [calcula_stress_embeddings(emb,r.embedding_)]
#         stress_original += [r.stress_]


    emb_one_out = emb[:p]+emb[p+1:]
    for i in range(5):
        r =MDS(n_components=d,max_iter=3000,n_init=10).fit(emb_one_out)
        stress_one_out += [calcula_stress_embeddings(emb_one_out,r.embedding_)]
#         stress_one_out += [r.stress_]
    return np.mean(stress_original)-np.mean(stress_one_out)

In [291]:
def get_emotion_embedding():
    return get_emb_concat()

def get_final_embs(dimensions = 200):
    embs = get_clean_embeddings()
    embs += [get_emb_soma()]
    embs += [get_emb_concat()]


    mds = MDS(n_components=dimensions)
    #Trata embeddings, filtrando somente os sentimentos principais e colocando no numero minimo de dimensoes
    embeddings = []
    for i in range(len(embs)):
        transformed_embedding = mds.fit_transform(embs[i])
        embeddings += [transformed_embedding]
    embs = embeddings

    for a in range(len(embs)):
        for b in range(len(embs)):
            embs[a],embs[b],r = procrustes(embs[a],embs[b])
    for i in range(200):
        a = np.random.randint(len(embs))
        b = np.random.randint(len(embs))
        embs[a],embs[b],r = procrustes(embs[a],embs[b])
    return embs

def calcula_stress_embeddings(d_original, d_proposta):
    d_original = distance_matrix(d_original,d_original)
    d_proposta = distance_matrix(d_proposta,d_proposta)
    a = np.sum((d_original-d_proposta)**2)
    b = np.sum(d_original**2)
    return np.sqrt(a/b)/4

def calcula_stress(d_original, d_proposta):
    a = np.sum((d_original-d_proposta)**2)
    b = np.sum(d_original**2)
    return np.sqrt(a/b)/4

def get_emb_soma():
    embeddings = get_clean_embeddings()
    sentiments = get_sentiments()
    assemble_embeddings_soma = np.zeros_like(embeddings[0])
    for i in embeddings:
        for c in range(len(sentiments)):
            assemble_embeddings_soma[c] = assemble_embeddings_soma[c] + i[c]
    return assemble_embeddings_soma

def get_emb_concat():
    embeddings = get_clean_embeddings()
    sentiments = get_sentiments()
    assemble_embeddings_concat = []

    for c in range(len(sentiments)):
        tmp = np.array([])
        for i in embeddings:
            tmp = np.concatenate((tmp,i[c]))
        assemble_embeddings_concat += [tmp]
    return assemble_embeddings_concat


def get_emb_media():
    #média dos vetores e retirando outliers
    embeddings = get_clean_embeddings()
    sentiments = get_sentiments()
    assemble_embeddings_media = np.zeros_like(embeddings[0])
    vec = []
    dist = []
    for c in range(len(sentiments)):
        tmp = np.array([])
        vec = []
        for i in embeddings:
            vec += [i[c]]

        centroid = np.zeros_like(vec[0])
        for i in vec:
            centroid = centroid + i
        centroid = centroid/len(vec)

        dist = []
        for i in vec:
            dist += [np.linalg.norm(i-centroid)]

        limitante_distancia_para_corte = np.max(dist)

        counter = 0
        for n,i in enumerate(embeddings):
            if dist[n]!=limitante_distancia_para_corte:
                assemble_embeddings_media[c] += i[c] 
                counter+=1

        assemble_embeddings_media[c] = assemble_embeddings_media[c]/counter
    return assemble_embeddings_media


def get_clean_embeddings():
#Seleciona somente os vetores principais das nossas embeddings
    sentiments = get_sentiments()
    raw_embeddings = []
    dimensions_size = []
    for i in range(4):
        tmp = get_vectors_embedding(i)
        raw_embeddings += [tmp]
        dimensions_size+=[len(get_sentiment_vector(tmp,sentiments[0]))]
#Faz o corte para o menor tamanho de dimensão
    n_dimensions = np.min(dimensions_size)
    # n_dimensions = 10
    mds = MDS(n_components=n_dimensions)
#Trata embeddings, filtrando somente os sentimentos principais e colocando no numero minimo de dimensoes
    embeddings = []
    for i in range(4):
        tmp_embedding = []
        for s in sentiments:
            tmp_embedding += [get_sentiment_vector(raw_embeddings[i],s)]
        transformed_embedding = mds.fit_transform(tmp_embedding)
        embeddings += [transformed_embedding]
    # Calcula erro entre nossos modelos
    for a in range(len(embeddings)):
        for b in range(len(embeddings)):
            embeddings[a],embeddings[b],r = procrustes(embeddings[a],embeddings[b])
    return embeddings

def print_anotado(emb):
    fig, ax = plt.subplots(figsize=(15,15))
    ax.scatter(emb[:,0], emb[:,1])
    n = get_sentiments()
    
    for i, txt in enumerate(n):
        ax.annotate(" "+txt, (emb[:,0][i], emb[:,1][i]))

def print_anotado2(emb):
    fig, ax = plt.subplots(figsize=(15,15))
    for e in emb:
        ax.scatter(e[:,0], e[:,1])
        n = get_sentiments()

        for i, txt in enumerate(n):
            ax.annotate(txt, (e[:,0][i], e[:,1][i]))

def print_2d(embs):
    res = []
    mds_2 = MDS(n_components=2)
    for e in embs:
        res += [mds_2.fit_transform(e)]
    for p in res:
        plt.scatter(p[:,0],p[:,1])

def get_sentiment_vector(embedding, sentiment):
    return embedding[sentiment]["vectors"][sentiment]

def get_vectors_embedding(n):
    files = os.listdir("models")
    model_file_name = []
    for file in files:
        model_file_name += [file]
    escolhidos = [model_file_name[14],model_file_name[5],model_file_name[7],model_file_name[3]]
    with open("models/"+escolhidos[n], 'rb') as handle:
        b = pickle.load(handle)
    return b

def get_sentiments_old():
    sentiments = np.load("./sentiments_list.npy")
    return sentiments
def get_sentiments():
    sentiments = np.load("./sentiments_list.npy")
    sentiments = np.concatenate((sentiments[:26],sentiments[27:]))
    return sentiments